In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import keras
from keras.datasets import mnist 
from keras.models import Sequential 
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#get data and pre process
(X_train,y_train),(X_test,y_test)= mnist.load_data()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [3]:
def plot_input_img(i):
    plt.imshow(X_train[i], cmap='binary')
    plt.title(y_train[i])
    plt.show

In [4]:
for i in range(10):
    plot_input_img(i)

In [5]:
#pre process image
#normalising the image to [0,1] range
X_train= X_train.astype(np.float32)/255
X_test= X_test.astype(np.float32)/255

#expand dimensions
X_train=np.expand_dims(X_train, -1)
X_test=np.expand_dims(X_test, -1)

#convert classes to one hot vectors
from tensorflow.keras.utils import to_categorical

y_train=tf.keras.utils.to_categorical(y_train)
y_test=tf.keras.utils.to_categorical(y_test)

In [6]:
model = Sequential()
model.add(Conv2D(32,(3,3), input_shape=(28,28,1),activation='relu'))
model.add(MaxPool2D(2,2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))

model.add(Flatten())
model.add(Dropout(0.25))#prevents overfitting
model.add(Dense(10, activation="softmax"))

In [7]:
model.summary()

In [8]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics = ['accuracy'])

In [9]:
#callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint

#EarlyStopping

es=EarlyStopping(monitor='val_acc', min_delta=0.01, patience= 4, verbose=1)

#Model Check Point

mc= ModelCheckpoint("./bestmodel.h5", monitor="val_acc", verbose=1, save_best_only = True)
model.save("bestmodel.h5")
cb=[es,mc]

In [10]:
#MODEL TRAINING

his= model.fit(X_train,y_train,epochs=50, validation_split=0.3)

In [13]:
model_S= keras.models.load_model('./bestmodel.h5')

In [14]:
score =model_S.evaluate(X_test,y_test)

print(f"the model accuracy is {score[1]}")